In [ ]:
# Creates a duct curved duct
def mapc2p_(xc, yc):

 
    p_centers = []
    
    mx, my = np.shape(xc)
    
    #Xp_centers = np.zeros((my, mx))
    #Yp_centers = np.zeros((my, mx))
    
    xp_edge_nodes = np.zeros((my+1, mx+1))
    yp_edge_nodes = np.zeros((my+1, mx+1))
    
    def Transfinite_Interpolation(x, y, Nx, Ny):
        
        NP_ZEROS = np.zeros

        x1 = NP_ZEROS((Ny, Nx))
        y1 = NP_ZEROS((Ny, Nx))

        for i in range(0, Nx):
            for j in range(0, Ny):
                phi0 = i/(Nx-1)
                phi1 = (Nx-i-1)/(Nx-1)
                x1[j,i] = phi0*x[j, Nx-1] + phi1*x[j,0]
                y1[j,i] = phi0*y[j, Nx-1] + phi1*y[j,0]

        x2 = NP_ZEROS((Ny, Nx))
        y2 = NP_ZEROS((Ny, Nx))

        for i in range(0, Nx):
            for j in range(0, Ny):
                psi0 = j/(Ny-1)
                psi1 = (Ny-j-1)/(Ny-1)
                x2[j,i] = psi0*(x[Ny-1,i] - x1[Ny-1,i]) + psi1*(x[0,i] - x1[0,i])
                y2[j,i] = psi0*(y[Ny-1,i] - y1[Ny-1,i]) + psi1*(y[0,i] - y1[0,i])

        for i in range(0, Nx):
            for j in range(0, Ny):
                x[j,i] = x1[j,i] + x2[j,i]
                y[j,i] = y1[j,i] + y2[j,i]            
            
        return x, y
    
    def Circular_Arc_Coordinates(x, y, R, theta, n):
        Cx = x + R*np.cos(np.pi/2 - theta)
        Cy = y - R*np.sin(np.pi/2 - theta)
    
        angles = np.linspace(theta, 0, n)
        x_arc_coordinates = np.zeros(n)
        y_arc_coordinates = np.zeros(n)
        for i in range(0, n):
            x_arc_coordinates[i] = Cx + R*np.cos(np.pi/2+angles[i])
            y_arc_coordinates[i] = Cy + R*np.sin(np.pi/2+angles[i])

        return x_arc_coordinates, y_arc_coordinates
    
    xp_Top, yp_Top = Circular_Arc_Coordinates(0., 0.1, 1, np.radians(20), mx+1)
    #plt.plot(xp_Top, yp_Top, 'k-') 
    
    xp_Bottom, yp_Bottom = Circular_Arc_Coordinates(0., 0., 1, np.radians(20), mx+1)
    #plt.plot(xp_Bottom, yp_Bottom, 'k-') 
    
    xp_Left = np.linspace(xp_Top[0], xp_Bottom[0], my+1)
    yp_Left = np.linspace(yp_Top[0], yp_Bottom[0], my+1)
    
    xp_Right = np.linspace(xp_Top[-1], xp_Bottom[-1], my+1)
    yp_Right = np.linspace(yp_Top[-1], yp_Bottom[-1], my+1)
    
    xp_edge_nodes[:,0] = xp_Left
    xp_edge_nodes[:,-1] = xp_Right
    xp_edge_nodes[0,:] = xp_Top
    xp_edge_nodes[-1,:] = xp_Bottom
    
    yp_edge_nodes[:,0] = yp_Left
    yp_edge_nodes[:,-1] = yp_Right
    yp_edge_nodes[0,:] = yp_Top
    yp_edge_nodes[-1,:] = yp_Bottom
    
    XP_edge_nodes, YP_edge_nodes = Transfinite_Interpolation(xp_edge_nodes, yp_edge_nodes, mx+1, my+1)
    
    XP_centers = XP_edge_nodes[:, 0:-1] + 0.5 * (XP_edge_nodes[:, 1:] - XP_edge_nodes[:, 0:-1])
    XP_Centers = 0.5 * (XP_centers[0:-1, :] + XP_centers[1:, :])
    
    YP_centers = YP_edge_nodes[1:, :] + 0.5* (YP_edge_nodes[0:-1, :] - YP_edge_nodes[1:, :])
    YP_Centers = 0.5 * (YP_centers[:, 0:-1] + YP_centers[:, 1:,])
    
    p_centers.append(XP_Centers.transpose())
    p_centers.append(YP_Centers.transpose())
    
    return p_centers 


# auxilliary array

def setaux(state, nx, ny):

    # Physical Domain Nodes
    xp_nodes, yp_nodes = state.grid.p_nodes

    # arrays consisting of the vertical edge normal vectors (n_VEdges) and the horizontal edge normal vectors (n_HEdges) 
    n_VEdges = np.zeros((2, nx, ny))
    n_HEdges = np.zeros((2, nx, ny))

    # Rotation Matrix: Rorates slope vector through 90deg to obtain the normal vector
    R = np.zeros((2,2))
    R[0,0] = np.cos(np.pi/2)
    R[0,1] = np.cos(np.pi/2 + np.pi/2)
    R[1,0] = np.cos(np.pi/2 - np.pi/2)
    R[1,1] = np.cos(np.pi/2)

    # computation of the outward pointing normal vectors
    
    for i in range(0, nx):
        for j in range(0, ny):
            u = np.zeros((2,1))
            u[0] = xp_nodes[i, j+1] -  xp_nodes[i ,j]
            u[1] = yp_nodes[i, j+1] -  yp_nodes[i ,j]
            n = R @ u
            n_unit = n/np.sqrt(n[0]**2 + n[1]**2)
            n_VEdges[0, i, j] = n_unit[0]
            n_VEdges[1, i, j] = n_unit[1]
    
    for i in range(0, nx):
        for j in range(0, ny):
            u = np.zeros((2,1))
            u[0] = xp_nodes[i+1, j] -  xp_nodes[i ,j]
            u[1] = yp_nodes[i+1 , j] -  yp_nodes[i ,j]
            n = R @ u
            n_unit = n/np.sqrt(n[0]**2 + n[1]**2)
            n_HEdges[0, i, j] = n_unit[0]
            n_HEdges[1, i, j] = n_unit[1]


    # Computation of the Length Ratios

    xc_nodes, yc_nodes = state.grid.c_nodes

    gamma_VEdges = np.zeros((nx, ny))
    gamma_HEdges = np.zeros((nx, ny))


    for i in range(0, nx):
        for j in range(0, ny):
            hp = np.sqrt((yp_nodes[i, j+1]- yp_nodes[i, j])**2 + (xp_nodes[i, j+1]- xp_nodes[i, j])**2 )
            hc = np.sqrt((yc_nodes[i, j+1]- yc_nodes[i, j])**2 + (xc_nodes[i, j+1]- xc_nodes[i, j])**2 )
            gamma_VEdges[i, j] = hp/hc

    for j in range(0, ny):
        for i in range(0, nx):
            wp = np.sqrt((yp_nodes[i+1, j]- yp_nodes[i, j])**2 + (xp_nodes[i+1,j]- xp_nodes[i, j])**2 )
            wc = np.sqrt((yc_nodes[i+1, j]- yc_nodes[i, j])**2 + (xc_nodes[i+1, j]- xc_nodes[i, j])**2 )
            gamma_HEdges[i, j] = wp/wc

    # Computation of the capacity (area ratios)
    capa = np.zeros((nx, ny))
    for i in range(0, nx):
        for j in range(0, ny):
            hp = np.sqrt((yp_nodes[i, j+1]- yp_nodes[i, j])**2 + (xp_nodes[i, j+1]- xp_nodes[i, j])**2 )
            hc = np.sqrt((yc_nodes[i, j+1]- yc_nodes[i, j])**2 + (xc_nodes[i, j+1]- xc_nodes[i, j])**2 )
            wp = np.sqrt((yp_nodes[i+1, j]- yp_nodes[i, j])**2 + (xp_nodes[i+1, j]- xp_nodes[i, j])**2 )
            wc = np.sqrt((yc_nodes[i+1, j]- yc_nodes[i, j])**2 + (xc_nodes[i+1, j]- xc_nodes[i, j])**2 )
            capa[i, j] = (hp*wp)/(hc*wc)

    # Populating the auxillary matrix
    aux = np.zeros((7, nx, ny))
    aux[0, :, :] = n_VEdges[0, :, :]
    aux[1, :, :] = n_VEdges[1, :, :]
    aux[2, :, :] = gamma_VEdges
    aux[3, :, :] = n_HEdges[0, :, :]
    aux[4, :, :] = n_HEdges[1, :, :]
    aux[5, :, :] = gamma_HEdges
    aux[6, :, :] =capa
    

    return aux



#print(aux)

# initial conditions. Assuming flow is initially stationary in the duct
def qinit(state):
    
    M = 0.5
    P = 101325.
    T = 300.
    rho = P/(287.*T)
    u = M * np.sqrt(1.4*287.*T) * np.cos(np.radians(20))
    v = M * np.sqrt(1.4*287.*T) * np.sin(np.radians(20))
    E = rho * (718.*T + (u**2 + v**2)/2.)
    
    T0 = T*(1. + ((1.4-1.)/2.)*M**2)
    P0 = P*(1. + ((1.4-1.)/2.)*M**2)**(1.4/(1.4-1))
    rho0 = rho*(1. + ((1.4-1.)/2.)*M**2)**(1./(1.4-1))
    u0 = 0.
    v0 = 0.
    E0 =  rho0 * (718.*T0 + (u0**2 + v0**2)/2.)

    
    state.q[0, :, :] = 0
    state.q[1, :, :] = rho0*u0
    state.q[2, :, :] = rho0*v0
    state.q[3, :, :] = E0
    
    
# Boundary condition on the LHS. Air is coming into the duct
def incoming_flow(state, dim, t, qbc, auxbc, num_ghost):
   
    M = 0.5
    P = 101325.
    T = 300.
    rho = P/(287.*T)
    u = M * np.sqrt(1.4*287.*T) * np.cos(np.radians(20))
    v = M * np.sqrt(1.4*287.*T) * np.sin(np.radians(20))
    E = rho * (718.*T + (u**2 + v**2)/2.)
    
    for i in range(num_ghost):
        qbc[0,i,...] = rho
        qbc[1,i,...] = rho*u
        qbc[2,i,...] = rho*v
        qbc[3,i,...] = E
        
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import absolute_import
import numpy as np
from clawpack import pyclaw
from clawpack import riemann
'''
import logging
logger = logging.getLogger('pyclaw')
logger.setLevel(logging.DEBUG)
'''

# Creating domain
nx = 100
ny = 30
x = pyclaw.Dimension(0, float(nx), nx, name='x')
y = pyclaw.Dimension(0, float(ny), ny, name='y')
domain = pyclaw.Domain([x, y])
domain.grid.mapc2p = mapc2p_

solver = pyclaw.ClawSolver2D(riemann.euler_mapgrid_2D)
#print(solver.num_waves)
#solver = pyclaw.ClawSolver2D(riemann.euler_4wave_2D)
domain.grid.num_ghost = solver.num_ghost
num_aux = 7

X, Y = domain.grid.p_centers
plt.scatter(X, Y, s = 15)
plt.axis('equal')

# Applying boundary conditions
solver.user_bc_lower = incoming_flow
solver.bc_lower[0] = pyclaw.BC.custom 
#solver.bc_lower[0] = pyclaw.BC.extrap
solver.bc_upper[0] = pyclaw.BC.extrap
solver.bc_lower[1] = pyclaw.BC.wall
solver.bc_upper[1] = pyclaw.BC.wall


solver.aux_bc_lower[0] = pyclaw.BC.extrap
solver.aux_bc_upper[0] = pyclaw.BC.extrap
solver.aux_bc_lower[1] = pyclaw.BC.wall
solver.aux_bc_upper[1] = pyclaw.BC.wall


state = pyclaw.State(domain,solver.num_eqn, num_aux)
state.problem_data['gamma'] = 1.4
state.aux[:,:,:] = setaux(state, nx, ny)
#print(setaux(state, nx, ny))
#print(state.aux)
#state.index_capa = 7
qinit(state)

claw = pyclaw.Controller()
claw.tfinal = 0.1
claw.num_output_times = 10
claw.keep_copy = True
#claw.output_format = None

claw.solution = pyclaw.Solution(state, domain)
claw.solver = solver

claw.run()


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

frame = claw.frames[0]
density = frame.q[0,:,:]
#print(density)
(vx,vy) = np.gradient(density)
vs = np.sqrt(vx**2 + vy**2)
x, y = frame.state.grid.p_centers

plt.pcolormesh(x, y, density, cmap='RdBu')
plt.colorbar()
plt.axis('image')

In [ ]:
from clawpack import riemann

solver = riemann.euler_mapgrid_2D